In [1]:
from urllib.request import urlopen
from urllib.parse import quote_plus as qp #문자 아스키코드로 변환
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import os

In [3]:
#인터넷 창 띄우기(chrome driver에 각자 개별 로그인)
driver = webdriver.Chrome()
url = 'https://www.instagram.com/'
driver.get(url)

In [16]:
#검색어 태그로 인터넷 창 이동
# 홈페이지 업데이트 했는지 class 명이 바뀜
class_name = '_aagw'
search = '바다랑대게' #검색어
baseurl = 'https://www.instagram.com/explore/tags/'
url = baseurl + qp(search)
driver.get(url)


TimeoutException: Message: 


In [18]:
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME,'_aagw')))
imgurl=[] #이미지 url 저장 list

In [32]:
class_name = '_aagt'
html = driver.page_source
soup = BeautifulSoup(html)
insta = soup.select('.'+class_name)

In [35]:
insta[0]['src']

'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/253182012_180105894240975_2339921964188325926_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=103&_nc_ohc=pEJ8JnWDi8gAX8s-4Vj&edm=AGyKU4gBAAAA&ccb=7-5&ig_cache_key=MjcwMTQwNzExODM3OTc2NzcxNg%3D%3D.2-ccb7-5&oh=00_AT8R9SXltLEfELBXAQF4XUk2JKbB12ph-dCtc6VeDgRpJQ&oe=62B7BCF5&_nc_sid=4cb768'

In [15]:
scroll_num = 100 #스크롤 횟수
scroll_pause_time=1.5 #스크롤 내릴때 이미지 로딩되는 시간 기다림
upscroll=-100
pic_num = 800 # 최대로 가져올 사진의 수

#스크롤 하기
#스크롤은 기다리기 귀찮으면 직접 한 후에 아래 코드 실행해도 됨
#인터넷 환경에 따라 이미지 로딩이 느릴때, 스크롤이 적게 실행될 수 있음
breaker = [] # 업데이트 5회이상 안될시, break


for i in range(scroll_num+1):

    #파싱
    html = driver.page_source
    soup = BeautifulSoup(html)
    insta = soup.select('.'+class_name)
    for pic in insta:
        imgurl.append(pic['src']) #이미지 주소 가져오기
        imgurl = (list(set(imgurl))) #이미지주소 중복 제거
    
    print('스크롤 {}번 실행, {}개 그림을 가져올 수 있습니다.'.format(i, len(list(set(imgurl)))))
    
    breaker.append(len(list(set(imgurl))))
    time.sleep(scroll_pause_time)
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script('scrollBy(0,{})'.format(upscroll))
    
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script('scrollBy(0,{})'.format(upscroll))
        if new_height == last_height:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script('scrollBy(0,{})'.format(upscroll))
        else:
            last_height = new_height
            continue
    if len(breaker)>5 and breaker[-5]==breaker[-1]: 
        print('더이상 update가 없습니다.')
        break
    if breaker[-1] >= pic_num:
        print(f'사진의 수가 {breaker[-1]}개로 최대개수인 {pic_num}개 이상입니다')
        break

NameError: name 'imgurl' is not defined

In [5]:
num = len(os.listdir('original_pictures')) #class 갯수
os.mkdir('./original_pictures/'+'{}_'.format(num+1)+search)  #폴더 생성
for i, pic in enumerate(imgurl):
    with urlopen(pic) as f: #날짜 시간 포함하여 이미지 저장
        with open('./original_pictures/'+'{}_'.format(num+1)+search+'/'+search+'_'+'{0:04d}'.format(i+1)+'_'+time.strftime('%y%m%d-%H%M')+'.jpg','wb') as h:
            img = f.read()
            h.write(img)
    print('이미지 저장 중 : {}/{}'.format(i+1,len(imgurl)), end='\r', flush=True)

In [44]:
driver.find_element_by_class_name('_ablz._aaoc')

InvalidSelectorException: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=102.0.5005.115)
